In [1]:
#SUPERcomputer Searching PeriodOgrams for Transits To EmeRge (SUPER-SPOTTER)

import numpy as np
import matplotlib.pyplot as plt
import batman
import scipy.signal as signal
import time
from tqdm import tqdm
from astropy import units as u
from astropy.io import ascii
from transitleastsquares import transitleastsquares

In /software/rhel7/lib64/python3.6/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: 
The text.latex.preview rcparam was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In /software/rhel7/lib64/python3.6/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: 
The mathtext.fallback_to_cm rcparam was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In /software/rhel7/lib64/python3.6/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: Support for setting the 'mathtext.fallback_to_cm' rcParam is deprecated since 3.3 and will be removed two minor releases later; use 'mathtext.fallback : 'cm' instead.
In /software/rhel7/lib64/python3.6/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: 
The validate_bool_maybe_none function was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In /software/rhel7/lib64/python3.6/site-packages/matplotlib/mpl-data/stylelib/_c

In [2]:
def binner(bins, time, flux, eflux):
    means, errors = [], []
    midpoints = [(bins[i] + bins[i+1])/2 for i in range(len(bins)-1)]
    for i in range(len(bins)-1):
        binned_mask = (time < bins[i+1])*(time > bins[i])
        if np.sum(binned_mask) < 5:
            mean = 1
        else:
            mean = np.average(flux[binned_mask], weights = 1/eflux[binned_mask]**2)
        means.append(mean)
        errors.append(np.std(flux[binned_mask])/np.sqrt(len(flux[binned_mask])))
    return means, errors, midpoints

In [3]:
data_file_name = 'Final_KARPA_Data_BRO2.csv' #The name of the input data file

input_periods = list(np.logspace(np.log10(3), np.log10(40), 100))[0:10] #The periodgrid
input_radii = list(np.linspace(0.5, 2, 11))                     #The radii grid
n_checks = 10                                                     #Amount of times to check each pixel

detections_file_name = 'detections_fake_KARPA2_set_0.csv'               #File name to save the number of detections file_name


#Define constants
r_earth = u.astrophys.earthRad.to(u.m)
r_star = 0.96*u.astrophys.R_sun.to(u.m)
m_star = 0.95*u.astrophys.M_sun.to(u.kg)
day_in_sec = u.day.to(u.second)
G = 6.6743*10**-11
r_jupiter = u.astrophys.jupiterRad.to(u.m)
resonances = [1./5.,1./4.,1./3., 1./2., 1., 2., 3., 4., 5.]

#Reading in the data
data = ascii.read(data_file_name)

mask = (data['col3'] < 0.05) #Removing measurements with high uncertainty
data = data[mask]

#Defining the times over which to produce fake data, the errors on the data and the transit parameters
times = data['col1']
transit_params = [2, 5, 10, m_star, r_star, 90, 0, 90]
e_flux = data['col3']


In [4]:
def transit_time(period):
    a = float(((((period*day_in_sec)**2)*G*m_star)/(4*np.pi**2))**(1./3.))/r_star 
    return period/(np.pi*a)

#Generating noise and fake data
def noise_generator(times, std):
    return np.random.randn(times.size)*std #Generating Gaussian noise with a sigma of choice
       
def fake_data_generator(times, noise):
    data = np.ones_like(times) 
    data += noise
    return data

#Generating a fake transit
def transit_generator(times, r_planet, t0, per, m_star, r_star, inc, ecc, w):
    params = batman.TransitParams()              #object to store transit parameters
    params.t0 = float(t0)                        #time of inferior conjunction
    params.per = float(per)                      #orbital period
    params.rp = float(r_planet*r_jupiter)/r_star #planet radius (in units of stellar radii)
    params.a = float(((((per*day_in_sec)**2)*G*m_star)/(4*np.pi**2))**(1./3.))/r_star   #semi-major axis (in units of stellar radii)
    params.inc = float(inc)                      #orbital inclination (in degrees)
    params.ecc = float(ecc)                      #eccentricity
    params.w = float(w)                          #longitude of periastron (in degrees)
    params.limb_dark = "nonlinear"               #limb darkening model "nonlinear"
    params.u = [1.2, -0.47, -0.22, 0.24]         #limb darkening coefficients [u1, u2, u3, u4] for T4500 log4.0, standard = [0.5, 0.1, 0.1, -0.1] 
    #print("The transit takes", params.per/(np.pi*params.a), "Days")

    t = times #times at which to calculate light curve 
    m = batman.TransitModel(params, t)    #initializes model
    return m.light_curve(params)

def tls(times, transit_flux, yerr, min_p, max_p, plot, stats): #tls 
    model = transitleastsquares(times, transit_flux, yerr)
    #Stellar parameters from Mentel 2018
    results = model.power(transit_depth_min = 0.001, use_threads = 96, R_star = 0.96, R_star_min = 0.81, R_star_max = 1.11, M_star = 0.95, M_star_min = 0.85, M_star_max = 1.05, period_min = min_p, period_max = max_p, oversampling_factor = 3, show_progress_bar = False)

    #-----------------------------------------------------------------------------
    #plt.plot(results.periods, results.power, c='black')
    #fig = plt.gcf()
    #fig.set_size_inches(9, 6)
    #plt.savefig("GB_no_tranist_added_SDE.png")
    #plt.show()
    #-----------------------------------------------------------------------------
    
    #Picking out the three most likely periods, excluding possible insignificantly different values
    picker = 1
    top_4_SDE = [0]
    while len(top_4_SDE) < 4:
        if len(top_4_SDE) == 3:
            if abs((results.periods[list(results.power).index(sorted(list(results.power))[-picker])]-results.periods[list(results.power).index(top_4_SDE[-1])])/(results.periods[list(results.power).index(top_4_SDE[-1])])) < 0.05 or abs((results.periods[list(results.power).index(sorted(list(results.power))[-picker])]-results.periods[list(results.power).index(top_4_SDE[-2])])/(results.periods[list(results.power).index(top_4_SDE[-2])])) < 0.05:
                pass
            else:
                top_4_SDE.append(sorted(list(results.power))[-picker])
        elif len(top_4_SDE) == 1:
            top_4_SDE.append(sorted(list(results.power))[-1])
        else:
            if abs((results.periods[list(results.power).index(sorted(list(results.power))[-picker])]-results.periods[list(results.power).index(top_4_SDE[-1])])/(results.periods[list(results.power).index(top_4_SDE[-1])])) < 0.05:
                pass
            else:
                top_4_SDE.append(sorted(list(results.power))[-picker])
        picker += 1
        
    top_3_SDE = top_4_SDE[1:]
    top_3_periods = [results.periods[list(results.power).index(top_3_SDE[0])], results.periods[list(results.power).index(top_3_SDE[1])], results.periods[list(results.power).index(top_3_SDE[2])]]
       
    print("SDE values: ", top_3_SDE)
    print("Period values: ", top_3_periods)
    #bins = np.linspace(0, top_3_periods[0], int(5*top_3_periods[0]/transit_time(top_3_periods[0])))
    #means, errors, midpoints = binner(bins, times%top_3_periods[0], transit_flux, yerr)
    #plt.errorbar(midpoints, means, errors, c='b', fmt = '.')
    #plt.axhline(y=1, linestyle = '--', c='grey')
    #plt.axvline(x=transit_params[1], linestyle = '--', c='r', alpha = 0.3)
    #plt.title("Binned Folded data")
    #plt.savefig("TESS_no_transit_added_binned.png")
    #plt.show()
    
    return top_3_periods, top_3_SDE, results

def retrieved_condition(retrieved_p, inserted_p, sde): #A function that determines whether a fake transit was retrieved
    print("Retrieved periods: ", retrieved_p)
    print("SDE: ", sde)
    for i in range(3):
        if sde[i] > 6 and abs(retrieved_p[i]-inserted_p)/(inserted_p) < 0.05: 
            return True, i
    return False, -1

def iterator(periods, radii, n_checks, stats): #Function iterating over different periods and radii
        sdes, detections = np.zeros((n_checks, len(radii), len(periods))), np.zeros((n_checks, len(radii), len(periods)))  #Array to store scores
        start = time.time()
        for i in tqdm(periods):
            for j in tqdm(radii):
                retrieved = 0 #Counter for succesful iterations
                transit_params[0], transit_params[2] = j, i #Setting transit parameters to a certain period, radius
                for k in tqdm(range(n_checks)): #testing parameter set 'n_checks' amount of times
                    print("Radius below is: ", j)
                    print("Period below is: ", '{0:.3f}'.format(i))
                    phase = np.random.uniform(0, i) #setting a random phase diff
                    transit_params[1] = phase
                    
                    flux = fake_data_generator(times, noise_generator(times, np.std(data['col2']))) #Creating fake transit flux
                    results = main(times, flux, e_flux, 3, i*2.1, False, stats, True, transit_params) #Looking for transit
                    detected, number = retrieved_condition(results[0], i, results[1])
                    if detected:
                        sdes[k, radii.index(j), periods.index(i)] = results[1][number]
                        detections[k, radii.index(j), periods.index(i)] = 1
                    else:
                        sdes[k, radii.index(j), periods.index(i)] = 0
                        detections[k, radii.index(j), periods.index(i)] = 0
                    
                    
                np.savetxt(detections_file_name, np.sum(detections, axis=0), delimiter=',')
            
        return sdes, detections 

def start(periods, radii, n_checks, stats=False):
    return iterator(periods, radii, n_checks, stats)

def main(times, transit_flux, yerr, min_p, max_p, plot, stats, add_transit, transit_params):
    if add_transit:

         #Building a dataset including a fake transit

        #transit_params[0], transit_params[1], transit_params[2] = 2, 1.91, 3.82

        transit_flux += (transit_generator(times, *transit_params)-1)

        #sig2_mask = (transit_flux < 1+2*np.std(transit_flux))*(transit_flux > 1-2*np.std(transit_flux))
        #times, transit_flux, yerr = times[sig2_mask], transit_flux[sig2_mask], yerr[sig2_mask]

        #----------------------------------------------------------------------------------------------------
        #plt.scatter(times, (transit_generator(times, *transit_params)-1), s=2)
        #plt.title("de transit")
        #plt.show()


        #bins = np.linspace(0, transit_params[2], int(3*transit_params[2]/transit_time(transit_params[2])))
        #means, errors, midpoints = binner(bins, times%transit_params[2], transit_flux, yerr)
        #plt.errorbar(midpoints, means, errors, c='b', fmt = '.')
        #plt.axhline(y=1, linestyle = '--', c='grey')
        #plt.axvline(x=transit_params[1], linestyle = '--', c='r', alpha = 0.3)
        #plt.title("Binned Folded data")
        #plt.savefig("Binned_post_transit_sig_clipped_flux.png")
        #plt.show()
        #----------------------------------------------------------------------------------------------------

    results = tls(times, transit_flux, yerr, min_p, max_p, plot, stats)
    return results


In [5]:
heat_map_data = start(input_periods, input_radii, n_checks)
print('Finished')

  0%|          | 0/11 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

Radius below is:  0.5
Period below is:  3.000
Transit Least Squares TLS 1.0.24 (21 November 2019)
Creating model cache for 23 durations
Searching 9843 data points, 13116 periods from 3.0 to 6.3 days
Using all 96 CPU threads


/home/barmentloo/.local/lib/python3.6/site-packages/transitleastsquares/transit.py:157: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  lc_arr = numpy.array(lc_arr)
/software/rhel7/lib64/python3.6/site-packages/numpy/core/_methods.py:234: RuntimeWarning: Degrees of freedom <= 0 for slice
  keepdims=keepdims)
/software/rhel7/lib64/python3.6/site-packages/numpy/core/_methods.py:195: RuntimeWarning: invalid value encountered in true_divide
  arrmean, rcount, out=arrmean, casting='unsafe', subok=False)
/software/rhel7/lib64/python3.6/site-packages/numpy/core/_methods.py:226: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/home/barmentloo/.local/lib/python3.6/site-packages/transitleastsquares/main.py:404: UserWarning: 1224 of 

SDE values:  [7.442631508327715, 6.041523744133051, 6.038446060135804]
Period values:  [5.77275603634179, 3.483768610362998, 6.068929945109428]
Retrieved periods:  [5.77275603634179, 3.483768610362998, 6.068929945109428]
SDE:  [7.442631508327715, 6.041523744133051, 6.038446060135804]
Radius below is:  0.5
Period below is:  3.000
Transit Least Squares TLS 1.0.24 (21 November 2019)
Creating model cache for 23 durations
Searching 9843 data points, 13116 periods from 3.0 to 6.3 days
Using all 96 CPU threads


/home/barmentloo/.local/lib/python3.6/site-packages/transitleastsquares/transit.py:157: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  lc_arr = numpy.array(lc_arr)
/home/barmentloo/.local/lib/python3.6/site-packages/transitleastsquares/main.py:404: UserWarning: 1513 of 1518 transits without data. The true period may be twice the given period.
  warnings.warn(text)


 20%|██        | 2/10 [06:05<25:15, 189.40s/it]

...

SDE values:  [73.00341252712795, 30.841457843005276, 13.761170146914633]
Period values:  [3.7965420436145507, 7.592974247785764, 4.745649041407933]
Retrieved periods:  [3.7965420436145507, 7.592974247785764, 4.745649041407933]
SDE:  [73.00341252712795, 30.841457843005276, 13.761170146914633]
Radius below is:  2.0
Period below is:  3.797
Transit Least Squares TLS 1.0.24 (21 November 2019)
Creating model cache for 24 durations
Searching 9843 data points, 16645 periods from 3.0 to 7.972 days
Using all 96 CPU threads




 90%|█████████ | 9/10 [29:44<03:20, 200.49s/it]

SDE values:  [52.67751715507996, 21.156652622691134, 12.887194794804929]
Period values:  [3.7965420436145507, 7.592974247785764, 5.6946632035635405]
Retrieved periods:  [3.7965420436145507, 7.592974247785764, 5.6946632035635405]
SDE:  [52.67751715507996, 21.156652622691134, 12.887194794804929]
Radius below is:  2.0
Period below is:  3.797
Transit Least Squares TLS 1.0.24 (21 November 2019)
Creating model cache for 24 durations
Searching 9843 data points, 16645 periods from 3.0 to 7.972 days
Using all 96 CPU threads




100%|██████████| 10/10 [56:50:12<00:00, 21155.92s/it] 

SDE values:  [57.3206039346357, 27.382199862446516, 11.427159144272062]
Period values:  [3.7965420436145507, 7.592974247785764, 4.745649041407933]
Retrieved periods:  [3.7965420436145507, 7.592974247785764, 4.745649041407933]
SDE:  [57.3206039346357, 27.382199862446516, 11.427159144272062]
Finished
